![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## This notebook is an example of how to pipeline two models. 
This notebook is an example of how to use DeGirum PySDK to do AI inference of a graphical file using 
two AI models: license plate detection and license recognition. The license plate detection model 
is run on the image and the results are then processed by the license recognition model, 
one license plate at a time. Combined result is then displayed.
This script uses PIL as image processing backend.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option.

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

When running this notebook in Google Colab, the cloud API access token should be stored in a user secret named `DEGIRUM_CLOUD_TOKEN`.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify where do you want to run your inferences

In [ ]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# lp_det_model_name: name of the model for license plate detection
# lp_ocr_model_name: name of the model for license plate OCR
# video_source: video source for inference
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
hw_location = "@cloud"
lp_det_model_zoo_url = "degirum/public"
lp_det_model_name = "yolo_v5s_lp_det--512x512_quant_n2x_orca1_1"
lp_ocr_model_zoo_url = "degirum/public"
lp_ocr_model_name = "yolo_v5s_lp_ocr--256x256_quant_n2x_orca1_1"
image_source = "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/Car.jpg"

#### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg, degirum_tools

# Load license plate detection and license plate OCR models
lp_det_model=dg.load_model(
    model_name=lp_det_model_name,
    inference_host_address=hw_location,
    zoo_url=lp_det_model_zoo_url,
    token=degirum_tools.get_token(),
)
lp_ocr_model=dg.load_model(
    model_name=lp_ocr_model_name,
    inference_host_address=hw_location,
    zoo_url=lp_ocr_model_zoo_url,
    token=degirum_tools.get_token(),
)

# Create a compound cropping model with 50% crop extent
crop_model = degirum_tools.CroppingAndClassifyingCompoundModel(
    lp_det_model,
    lp_ocr_model,
    50.0
)

# Detect license plate boxes
inference_result = crop_model(image_source)

# display combined results
with degirum_tools.Display("License Plates") as display:
    display.show_image(inference_result)
print(inference_result)